In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
dir = os.getcwd()

In [4]:
df = pd.read_csv(dir+'/data/merged.csv')

In [5]:
df.head()

,date,bsi_inv_big,bsi_inv_small,3y,10y,shipment,inven,ipi,employ,durable,wti,dubai,iron,copper,gold
0,1975-01-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1975-02-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1975-03-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1975-04-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1975-05-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df1 = df.drop(columns=['wti', 'iron', 'copper', 'gold'])

In [7]:
#날짜를 인덱스로 설정
df1.set_index('date', inplace=True)

In [8]:
df1

,bsi_inv_big,bsi_inv_small,3y,10y,shipment,inven,ipi,employ,durable,dubai
date,,,,,,,,,,
1975-01-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN
1975-02-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN
1975-03-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN
1975-04-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN
1975-05-01,NaN,NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-04-01,94.0,86.0,2.395,2.658,101.8,104.0,113.8,63.1,99.1,68.49
2025-05-01,94.0,86.0,2.331,2.710,99.8,104.0,112.5,62.9,100.5,63.61
2025-06-01,97.0,86.0,2.441,2.842,101.6,104.2,113.8,62.8,98.9,69.36


In [9]:
df1.dropna(inplace=True) #결측치 제거

In [10]:
#문자로 된 자료 숫자로 바꿔주기
for a in df1.columns:
    df1[a] = df1[a].astype('float')

In [11]:
df1

,bsi_inv_big,bsi_inv_small,3y,10y,shipment,inven,ipi,employ,durable,dubai
date,,,,,,,,,,
2003-02-01,99.0,98.0,4.730,5.150,56.009,43.439,61.7,59.6,37.2,30.20
2003-03-01,95.0,95.0,4.780,5.200,56.346,44.196,62.3,59.5,36.4,27.52
2003-04-01,87.0,91.0,4.600,5.030,55.752,44.925,62.4,59.2,36.3,23.56
2003-05-01,91.0,91.0,4.270,4.680,55.680,44.799,62.4,59.2,35.9,24.32
2003-06-01,87.0,90.0,4.070,4.460,57.136,45.114,63.3,59.2,34.5,25.67
...,...,...,...,...,...,...,...,...,...,...
2025-02-01,91.0,87.0,2.611,2.830,99.900,106.600,113.4,63.0,110.6,76.32
2025-03-01,93.0,87.0,2.590,2.795,100.900,104.100,114.6,63.0,100.7,72.98
2025-04-01,94.0,86.0,2.395,2.658,101.800,104.000,113.8,63.1,99.1,68.49


In [12]:
#장단기 스프레드 // 출하지수-재고지수 스프레드
df1['y_spd'] = df1['10y'] - df1['3y']
df1['manu_spd'] = df1['shipment'] - df1['inven']
df1['manu_ratio'] = df1['shipment'] / df1['inven'] #스프레드하면 스케일이 약간 애매해져서, 양 지수에 로그취해서 빼줌

In [13]:
# 샘플기간 분리
df1_train = df1['2003-02-01':'2017-06-01']
df1_test = df1['2017-07-01':]

In [14]:
#필요없는거 추가로 제거
df2 = df1.drop(columns=['10y', '3y', 'shipment', 'inven'])

In [15]:
#설비투자전망지수 대기업+중소기업 평균 산출
df2['bsi_inv'] = (df2['bsi_inv_big'] + df2['bsi_inv_small']) / 2

In [16]:
#필요없는거 또 제거
df2.drop(columns=['bsi_inv_small', 'bsi_inv_big'], inplace=True)

In [17]:
df2.drop(columns='ipi', inplace=True)

In [18]:
df2

,employ,durable,dubai,y_spd,manu_spd,manu_ratio,bsi_inv
date,,,,,,,
2003-02-01,59.6,37.2,30.20,0.420,12.570,1.289371,98.5
2003-03-01,59.5,36.4,27.52,0.420,12.150,1.274912,95.0
2003-04-01,59.2,36.3,23.56,0.430,10.827,1.241002,89.0
2003-05-01,59.2,35.9,24.32,0.410,10.881,1.242885,91.0
2003-06-01,59.2,34.5,25.67,0.390,12.022,1.266480,88.5
...,...,...,...,...,...,...,...
2025-02-01,63.0,110.6,76.32,0.219,-6.700,0.937148,89.0
2025-03-01,63.0,100.7,72.98,0.205,-3.200,0.969260,90.0
2025-04-01,63.1,99.1,68.49,0.263,-2.200,0.978846,90.0


In [19]:
list_log = ['employ', 'durable', 'dubai', 'bsi_inv']
for a in list_log:
    df2[a+'_l'] = np.log(df2[a])

### 주성분 분석
- Test 데이터 + 1시점씩 늘리면서
- AR(1) 예측
- 예측한걸로 l1 필터링 적용해서 t+1 시점의 국면 예측

다양한 방식 시도
1. 원자료 그대로
1. 지수형 자료 로그변환
1. 출하지수-재고지수 스프레드: 원자료끼리 뺀 거
1. 출하지수-재고지수 스프레드: 둘다 로그취해서 뺀 거

In [20]:
df2.columns

Index(['employ', 'durable', 'dubai', 'y_spd', 'manu_spd', 'manu_ratio',
       'bsi_inv', 'employ_l', 'durable_l', 'dubai_l', 'bsi_inv_l'],
      dtype='object')

In [47]:
from sklearn.preprocessing import StandardScaler
x1 = ['employ', 'durable', 'dubai', 'y_spd', 'manu_spd', 'bsi_inv']

#PCA 함수: 데이터프레임을 인풋으로
#PCA 결과들을 리스트에 모아서 리턴해줌

def pca(x):
    '''
    데이터프레임을 인풋으로
    PCA 결과들을 딕셔너리에 모아서 리턴해줌
    딕셔너리 키 목록
    1. df_scaled: 표준화된 데이터
    2. cov_matrix: 표준화된 데이터의 공분산 행렬
    3. eigenvalues_df: eigenvalues
    4. eigenvectors_df: eigenvectors
    5. explained_variance_ratio_df: 분산 설명비율
    '''
    scaler = StandardScaler() # standard normal
    df_scaled = scaler.fit_transform(x) #데이터 표준화
    cov_matrix = np.cov(df_scaled, rowvar=False) #공분산 행렬 계산
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix) #아이겐벡터, 밸류 계산
    principal_components = df_scaled.dot(eigenvectors[:, 0])  # 첫 번째 주성분(PC1)

    # PC1 방향을 나타내는 벡터
    pc1_vector = eigenvectors[:, 0]

    #Explained Variance Ratio
    explained_variance_ratio = eigenvalues / np.sum(eigenvalues)

    #PCA 결과 저장
    index = []
    for i in range(0,len(x.columns)):
        index.append(f'PC{i+1}')

    cov_matrix_df = pd.DataFrame(cov_matrix, columns=x.columns, index=x.columns)
    eigenvalues_df = pd.DataFrame({'Eigenvalues': eigenvalues}, index=index)
    eigenvectors_df = pd.DataFrame(eigenvectors, columns=index,
                                index=x.columns)
    explained_variance_ratio_df = pd.DataFrame({'Explained Variance Ratio': explained_variance_ratio},
                                            index=index)
    result = {'df_scaled' : df_scaled,
              'cov_matrix' : cov_matrix,
              'eigenvalues' : eigenvalues_df,
              'eigenvectors' : eigenvectors_df,
              'explained_variance' : explained_variance_ratio_df}
    print(result.keys())
    return result

In [22]:
pca1 = pca(df2[['employ', 'durable', 'dubai', 'y_spd', 'manu_spd', 'bsi_inv']])

dict_keys(['df_scaled', 'cov_matrix', 'eigenvalues', 'eigenvectors', 'explained_variance'])


In [23]:
pca1['df_scaled']

array([[-0.72442167, -1.40470893, -1.58989313, -0.1189931 ,  0.29642427,
         0.67192389],
       [-0.81422325, -1.43814328, -1.69674955, -0.1189931 ,  0.2585572 ,
        -0.04573504],
       [-1.08362799, -1.44232257, -1.85464187, -0.08537358,  0.13927594,
        -1.27600748],
       ...,
       [ 2.41863364,  1.18227415, -0.06320199, -0.64681954, -1.0352343 ,
        -1.07096208],
       [ 2.23903048,  1.24078427, -0.25777636, -0.25683313, -1.21555367,
        -1.07096208],
       [ 2.1492289 ,  1.17391556, -0.02851352, -0.18287018, -1.07129818,
        -0.76339396]])

In [46]:
# PC1 시각화
'''
plt.figure(figsize=(8, 6))
plt.scatter(df2_scaled[:, 0], df2_scaled[:, 1], c=principal_components, cmap='viridis')
plt.colorbar(label='PC1')
plt.quiver(0, 0, pc1_vector[0], pc1_vector[1], angles='xy', scale_units='xy', scale=2, color='red', label='PC1 Direction')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Visualization of PC1 with PC1 Direction (Noisy Data)')
plt.legend()
plt.show()
'''

"\nplt.figure(figsize=(8, 6))\nplt.scatter(df2_scaled[:, 0], df2_scaled[:, 1], c=principal_components, cmap='viridis')\nplt.colorbar(label='PC1')\nplt.quiver(0, 0, pc1_vector[0], pc1_vector[1], angles='xy', scale_units='xy', scale=2, color='red', label='PC1 Direction')\nplt.xlabel('Feature 1')\nplt.ylabel('Feature 2')\nplt.title('Visualization of PC1 with PC1 Direction (Noisy Data)')\nplt.legend()\nplt.show()\n"

In [25]:
pca1['eigenvectors']

,PC1,PC2,PC3,PC4,PC5,PC6
employ,0.517859,0.119037,-0.534840,0.573729,-0.290145,-0.135090
durable,0.470130,0.065772,-0.176064,-0.628420,-0.291200,0.513755
dubai,0.036563,0.718987,0.008185,0.188886,0.528684,0.408005
y_spd,-0.377971,-0.290919,-0.016649,0.412996,-0.282897,0.722240
manu_spd,-0.539062,0.114723,-0.785739,-0.263170,0.051712,-0.083268
bsi_inv,-0.275705,0.605612,0.255378,-0.020523,-0.684695,-0.150913


In [26]:
#pca1의 주성분을 PC1, PC2로 설정
#표준화된 X 랑 eigenvector 곱하기
data_pc1 = pca1['df_scaled']@pca1['eigenvectors'][['PC1', 'PC2']]

In [27]:
data_pc1

,PC1,PC2
0,-1.393744,-0.846188
1,-1.241599,-1.374872
2,-0.998065,-2.289271
3,-1.095090,-2.000108
4,-1.009340,-2.244337
...,...,...
264,3.425842,-0.141257
265,3.017608,-0.090146
266,2.903827,-0.258955
267,2.781010,-0.550524


In [28]:
pca1['eigenvalues'].iloc[0]

Eigenvalues    3.108234
Name: PC1, dtype: float64

In [29]:
# 전체 eigenvalues의 합에서 각 컴포넌트의 eigenvalue의 비율만큼 가중치 부여
# weight = eigenvalue_i / sum(eigenvalues)
w0 = pca1['explained_variance']['Explained Variance Ratio'].iloc[0]
w1 = pca1['explained_variance']['Explained Variance Ratio'].iloc[1]
data_pc1['growth_index'] = data_pc1['PC1'] * w0 + data_pc1['PC2'] * w1

In [30]:
data_pc1

,PC1,PC2,growth_index
0,-1.393744,-0.846188,-0.922602
1,-1.241599,-1.374872,-0.971079
2,-0.998065,-2.289271,-1.065045
3,-1.095090,-2.000108,-1.045658
4,-1.009340,-2.244337,-1.060070
...,...,...,...
264,3.425842,-0.141257,1.734189
265,3.017608,-0.090146,1.535772
266,2.903827,-0.258955,1.436497
267,2.781010,-0.550524,1.303069


In [ ]:
data_pc1.index = df2.index #날짜 인덱스 지정해주고
data_pc1.to_csv(dir + '/data/pc_data1.csv') #저장하기

In [31]:
# PCA 두번째: 지수형 자료 로그변환
df2

,employ,durable,dubai,y_spd,manu_spd,manu_ratio,bsi_inv,employ_l,durable_l,dubai_l,bsi_inv_l
date,,,,,,,,,,,
2003-02-01,59.6,37.2,30.20,0.420,12.570,1.289371,98.5,4.087656,3.616309,3.407842,4.590057
2003-03-01,59.5,36.4,27.52,0.420,12.150,1.274912,95.0,4.085976,3.594569,3.314913,4.553877
2003-04-01,59.2,36.3,23.56,0.430,10.827,1.241002,89.0,4.080922,3.591818,3.159550,4.488636
2003-05-01,59.2,35.9,24.32,0.410,10.881,1.242885,91.0,4.080922,3.580737,3.191299,4.510860
2003-06-01,59.2,34.5,25.67,0.390,12.022,1.266480,88.5,4.080922,3.540959,3.245323,4.483003
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-01,63.0,110.6,76.32,0.219,-6.700,0.937148,89.0,4.143135,4.705920,4.334935,4.488636
2025-03-01,63.0,100.7,72.98,0.205,-3.200,0.969260,90.0,4.143135,4.612146,4.290185,4.499810
2025-04-01,63.1,99.1,68.49,0.263,-2.200,0.978846,90.0,4.144721,4.596129,4.226688,4.499810


In [32]:
pca2 = pca(df2[['employ_l', 'durable_l', 'dubai_l', 'y_spd', 'manu_spd', 'bsi_inv_l']])

dict_keys(['df_scaled', 'cov_matrix', 'eigenvalues', 'eigenvectors', 'explained_variance'])


In [52]:
pca1['eigenvectors']

,PC1,PC2,PC3,PC4,PC5,PC6
employ,0.517859,0.119037,-0.534840,0.573729,-0.290145,-0.135090
durable,0.470130,0.065772,-0.176064,-0.628420,-0.291200,0.513755
dubai,0.036563,0.718987,0.008185,0.188886,0.528684,0.408005
y_spd,-0.377971,-0.290919,-0.016649,0.412996,-0.282897,0.722240
manu_spd,-0.539062,0.114723,-0.785739,-0.263170,0.051712,-0.083268
bsi_inv,-0.275705,0.605612,0.255378,-0.020523,-0.684695,-0.150913


In [51]:
pca2['eigenvectors']

,PC1,PC2,PC3,PC4,PC5,PC6
employ_l,-0.524949,-0.075390,-0.166511,-0.579425,-0.329736,-0.496548
durable_l,-0.460661,-0.119075,0.529452,-0.126903,-0.252202,0.643104
dubai_l,-0.100255,-0.708709,0.356573,0.024069,0.528087,-0.284747
y_spd,0.390590,0.208453,0.714125,-0.033204,-0.349561,-0.413179
manu_spd,0.536858,-0.165446,-0.056614,-0.763356,0.111157,0.293491
bsi_inv_l,0.247773,-0.638000,-0.227224,0.252522,-0.643618,0.043848


In [ ]:
pca2['explained_variance'] #얘는 주성분 세번째까지 사용

,Explained Variance Ratio
PC1,0.505204
PC2,0.246419
PC3,0.125431
PC4,0.016637
PC5,0.066955
PC6,0.039355


In [35]:
#pca1의 주성분을 PC1, PC2, PC3로 설정
#표준화된 X 랑 eigenvector 곱하기
data_pc2 = pca2['df_scaled']@pca2['eigenvectors'][['PC1', 'PC2', 'PC3']]

In [37]:
# 전체 eigenvalues의 합에서 각 컴포넌트의 eigenvalue의 비율만큼 가중치 부여
# weight = eigenvalue_i / sum(eigenvalues)
w0 = pca2['explained_variance']['Explained Variance Ratio'].iloc[0]
w1 = pca2['explained_variance']['Explained Variance Ratio'].iloc[1]
w2 = pca2['explained_variance']['Explained Variance Ratio'].iloc[2]
data_pc2['growth_index'] = data_pc2['PC1'] * w0 + data_pc2['PC2'] * w1 + data_pc2['PC3'] * w2

In [49]:
data_pc1

,PC1,PC2,growth_index
0,-1.393744,-0.846188,-0.922602
1,-1.241599,-1.374872,-0.971079
2,-0.998065,-2.289271,-1.065045
3,-1.095090,-2.000108,-1.045658
4,-1.009340,-2.244337,-1.060070
...,...,...,...
264,3.425842,-0.141257,1.734189
265,3.017608,-0.090146,1.535772
266,2.903827,-0.258955,1.436497
267,2.781010,-0.550524,1.303069


In [38]:
data_pc2

,PC1,PC2,PC3,growth_index
0,1.588906,1.124677,-1.672952,0.870022
1,1.496276,1.753618,-1.613849,0.985621
2,1.323946,2.877689,-1.398007,1.202625
3,1.411566,2.538148,-1.530281,1.146630
4,1.344966,2.763026,-1.471890,1.175722
...,...,...,...,...
264,-3.290246,0.258805,0.299182,-1.560943
265,-2.956460,0.171443,0.022565,-1.448537
266,-2.840982,0.310009,0.061621,-1.351153
267,-2.693396,0.562489,0.332701,-1.180374


In [39]:
data_pc2.index = df2.index #날짜 인덱스 지정해주고
data_pc2.to_csv(dir + '/data/pc_data2.csv') #저장하기

In [41]:
# PCA 세번째: 지수형 자료 로그변환 + 출하지수-재고지수 로그변환 스프레드
pca3 = pca(df2[['employ_l', 'durable_l', 'dubai_l', 'y_spd', 'manu_ratio', 'bsi_inv_l']])

dict_keys(['df_scaled', 'cov_matrix', 'eigenvalues', 'eigenvectors', 'explained_variance'])


In [ ]:
pca3['explained_variance'] #얘도 세 번째까지 주성분 선택

,Explained Variance Ratio
PC1,0.512653
PC2,0.241353
PC3,0.129865
PC4,0.012909
PC5,0.066826
PC6,0.036395


In [43]:
#pca1의 주성분을 PC1, PC2, PC3로 설정
#표준화된 X 랑 eigenvector 곱하기
data_pc3 = pca3['df_scaled']@pca3['eigenvectors'][['PC1', 'PC2', 'PC3']]

In [44]:
# 전체 eigenvalues의 합에서 각 컴포넌트의 eigenvalue의 비율만큼 가중치 부여
# weight = eigenvalue_i / sum(eigenvalues)
w0 = pca3['explained_variance']['Explained Variance Ratio'].iloc[0]
w1 = pca3['explained_variance']['Explained Variance Ratio'].iloc[1]
w2 = pca3['explained_variance']['Explained Variance Ratio'].iloc[2]
data_pc3['growth_index'] = data_pc3['PC1'] * w0 + data_pc3['PC2'] * w1 + data_pc3['PC3'] * w2

In [45]:
data_pc3

,PC1,PC2,PC3,growth_index
0,1.942499,0.945402,-1.597290,1.016571
1,1.841855,1.593029,-1.562352,1.125819
2,1.651283,2.749855,-1.383610,1.330537
3,1.733910,2.397464,-1.489542,1.274089
4,1.699011,2.633016,-1.480796,1.314185
...,...,...,...,...
264,-3.219365,0.374814,0.102907,-1.546590
265,-2.942802,0.296116,-0.158776,-1.457787
266,-2.839197,0.437982,-0.120069,-1.365407
267,-2.660904,0.665122,0.180206,-1.180188


In [48]:
data_pc3.index = df2.index #날짜 인덱스 지정해주고
data_pc3.to_csv(dir + '/data/pc_data3.csv') #저장하기